In [1]:
import numpy


In [4]:
from collections import defaultdict
import xml.etree.ElementTree
import random

def load_data():
    prefix = "Fingerprint\\data\\"

    def load_filenames(filename):
        e = xml.etree.ElementTree.parse(prefix + filename).getroot()
        res = []
        print(e.find('fingerprints').keys())
        for fingerprint in list(e.find('fingerprints')):
            a = fingerprint.attrib
            res.append((a['file'], a['class']))
        return res

    def load_dataset(filelist):
        res = []
        graph_id = 0

        for fname, cls in filelist:
            e = xml.etree.ElementTree.parse(prefix + fname)
            graph = e.find('graph')

            nodes = dict()
            edges_connected_to_nodes = defaultdict(list)
            edge      = list()
            edge_info = dict()

            for child in list(graph):
                if child.tag == 'node':
                    node_id = child.attrib['id']
                    node_xy = [0, 0]
                    for node_attrib in list(child):
                        attrib_value = float(list(node_attrib)[0].text)

                        if node_attrib.attrib['name'] == 'x':
                            node_xy[0] = attrib_value
                        else:
                            node_xy[1] = attrib_value
                    nodes[node_id] = node_xy
                elif child.tag == 'edge':
                    edge_orient_angle = [0, 0]
                    for edge_attrib in list(child):
                        attrib_value = float(list(edge_attrib)[0].text)

                        if edge_attrib.attrib['name'] == 'orient':
                            edge_orient_angle[0] = attrib_value
                        elif edge_attrib.attrib['name'] == 'angle':
                            edge_orient_angle[1] = attrib_value

                    from_id = child.attrib['from']
                    to_id = child.attrib['to']
                    edge_weight = 1
                    edge_info[(from_id, to_id)] = [edge_weight ,edge_orient_angle]
                    edge.append((from_id, to_id))

            for node in nodes.keys():
                edges_connected_to_nodes[node] = ([], [])
                for item in edge:
                    if item[0] == node:  # outcome edge
                        edges_connected_to_nodes[node][0].append(item)
                    elif item[1] == node:  # income edge
                        edges_connected_to_nodes[node][1].append(item)

            if cls == "T":
                cls = "A"
            elif cls[0] == "T":
                cls = "A"
            else:
                cls = cls[0]

            if edges_connected_to_nodes:

                type = "Normal graph"

            else:

                type = "Empty graph"

            res.append((graph_id, nodes, edges_connected_to_nodes, edge_info, cls, type))
            graph_id += 1

        return res
    
    train_list = load_filenames('train.cxl')
    valid_list=load_filenames('valid.cxl')
    test_list=load_filenames('test.cxl')

    train_dataset = load_dataset(train_list)
    valid_dataset = load_dataset(valid_list)
    test_dataset = load_dataset(test_list)

    return train_dataset, valid_dataset, test_dataset

train, valid, test = load_data()


['base', 'classmodel', 'count']
['base', 'classmodel', 'count']
['base', 'classmodel', 'count']


In [5]:
import math
from collections import Counter

ALPHA = 0.5
ALPHA_ = 1 - ALPHA


def compute_edge_cost(e1, e2,w_e1):
    absdiff = abs((e1 * w_e1)- e2)
    return min(absdiff, 2 * math.pi - absdiff)


def HEC(a_edges, b_edges,a_edges_info,b_edges_info,TAU_E):

    a_edges_in = a_edges[1]
    a_edges_out = a_edges[0]
    b_edges_in = b_edges[1]
    b_edges_out = b_edges[0]

    a_costs_in  =dict()
    a_costs_out =dict()

    for item in a_edges_in :

        a_costs_in[item] = ALPHA_  *  TAU_E * a_edges_info[item][0]

    for item in a_edges_out :
        a_costs_out[item] = ALPHA_  *  TAU_E * a_edges_info[item][0]

    b_costs_in = [ALPHA_ * TAU_E for _ in b_edges_in]
    b_costs_out = [ALPHA_ * TAU_E for _ in b_edges_out]


    for x in range(len(a_edges_in)):

        id_1=a_edges_in[x]
        for y in range(len(b_costs_in)):

            id_2=b_edges_in[y]
            edge_substitution_cost = ALPHA_ * compute_edge_cost(a_edges_info[id_1][1][1], b_edges_info[id_2][1][1],a_edges_info[id_1][0])
            a_costs_in[id_1] = min(a_costs_in[id_1], edge_substitution_cost / 2)
            b_costs_in[y] = min(b_costs_in[y], edge_substitution_cost / 2)


    for i in range(len(a_edges_out)):

        id_1 = a_edges_out[i]
        for j in range(len(b_costs_out)):

            id_2 = b_edges_out[j]
            edge_substitution_cost = ALPHA_ * compute_edge_cost(a_edges_info[id_1][1][1], b_edges_info[id_2][1][1],a_edges_info[id_1][0])
            a_costs_out[id_1] = min(a_costs_out[id_1], edge_substitution_cost / 2)
            b_costs_out[j] = min(b_costs_out[j], edge_substitution_cost / 2)

    return  sum_elem(a_costs_out) + sum_elem(a_costs_in) + sum(b_costs_in) + sum(b_costs_out)


def HED(g1, g2, TAU_E, TAU_N):

    g1_nodes, g1_edges ,g1_edges_info = g1[1], g1[2] ,g1[3]
    g2_nodes, g2_edges ,g2_edges_info = g2[1], g2[2] ,g2[3]

    g1_node_costs = []
    g2_node_costs = []

    g1_node_names = list(g1_nodes.keys())
    g2_node_names = list(g2_nodes.keys())


    for n in g1_node_names:
        cost = TAU_N * ALPHA
        for x in g1_edges[n] :
            for y in range(len(x)):
                cost += ALPHA_ * TAU_E / 2
        g1_node_costs.append(cost)

    for n in g2_node_names:
        cost = TAU_N * ALPHA
        for x in  g2_edges[n] :
            for y in range(len(x)):
                cost += ALPHA_ * TAU_E / 2
        g2_node_costs.append(cost)

    for i in range(len(g1_node_names)):
        degree_i =0

        edge_connected_to_node_i = g1_edges[g1_node_names[i]]
        for item in g1_edges[g1_node_names[i]] :
            for edge in item :
                degree_i += 1

        for j in range(len(g2_node_names)):
            degree_j = 0

            edge_connected_to_node_j = g2_edges[g2_node_names[j]]
            for item in g2_edges[g2_node_names[j]]:
                for edge in item:
                    degree_j += 1

            edge_substitution_cost = HEC(edge_connected_to_node_i,edge_connected_to_node_j,g1_edges_info, g2_edges_info ,TAU_E)
            edge_substitution_cost = max(edge_substitution_cost,abs(degree_i - degree_j) * TAU_E * ALPHA_)
            g1_node_costs[i] = min(g1_node_costs[i], (edge_substitution_cost / 4))
            g2_node_costs[j] = min(g2_node_costs[j], (edge_substitution_cost / 4))

    return max(sum(g1_node_costs) + sum(g2_node_costs), abs(len(g1_node_names) - len(g2_node_names)) * ALPHA * TAU_N)


from collections import defaultdict
def sum_elem(dict ):

    total =0
    for elem in dict.values():
        total +=elem

    return total


In [10]:
from sklearn.model_selection import ParameterGrid
import operator


def getNeighbors(trainingSet, Instance, k, TAU_E, TAU_N):
    distances = []
    neighbor  = []

    for x in range(len(trainingSet)):
        dist = HED(trainingSet[x], Instance, TAU_E, TAU_N)
        distances.append((trainingSet[x][-2], dist))

    distances.sort(key=operator.itemgetter(1))
    for x in range(k):
        neighbor.append(distances[x][0])

    return neighbor


def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][0]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]


def getAccuracy(validSet, predictions):
    correct = 0
    for x in range(len(validSet)):
        if validSet[x][-2] == predictions[x]:
            correct += 1
    return (correct / float(len(validSet))) * 100


def find_optimal_params(train, valid):
    param_grid = {'TAU_E': [0.1, 0.4, 0.7, 1.0, 1.3, 1.6, 1.9, 2.2, 2.5, 10.0],
                  'TAU_N': [0.1, 0.4, 0.7, 1.0, 1.3, 1.6, 1.9, 2.2, 2.5, 10.0]
                  }

    grid = ParameterGrid(param_grid)

    best_hypers = None
    best_val = float('-inf')
    for params in grid:
        predictions = []
        for y in range(len(valid)):
            neighbors = getNeighbors(train, valid[y], k=1 , TAU_E=params['TAU_E'], TAU_N=params['TAU_N'])
            result = getResponse(neighbors)
            predictions.append(result)
        accuracy = getAccuracy(valid, predictions)
        if accuracy > best_val:
            best_val = accuracy
            best_hypers = params
    return best_hypers, best_val


In [7]:
train, valid, test = load_data()
dist = HED(train[13], train[22], TAU_E= 2.5 , TAU_N= 1.3)
dist

['base', 'classmodel', 'count']
['base', 'classmodel', 'count']
['base', 'classmodel', 'count']


1.3032114133974482

In [8]:
train, valid, test = load_data()
train

['base', 'classmodel', 'count']
['base', 'classmodel', 'count']
['base', 'classmodel', 'count']


[(0, {}, defaultdict(list, {}), {}, 'A', 'Empty graph'),
 (1, {}, defaultdict(list, {}), {}, 'A', 'Empty graph'),
 (2, {}, defaultdict(list, {}), {}, 'A', 'Empty graph'),
 (3, {}, defaultdict(list, {}), {}, 'A', 'Empty graph'),
 (4, {}, defaultdict(list, {}), {}, 'A', 'Empty graph'),
 (5, {}, defaultdict(list, {}), {}, 'A', 'Empty graph'),
 (6, {}, defaultdict(list, {}), {}, 'A', 'Empty graph'),
 (7, {}, defaultdict(list, {}), {}, 'A', 'Empty graph'),
 (8, {}, defaultdict(list, {}), {}, 'A', 'Empty graph'),
 (9, {}, defaultdict(list, {}), {}, 'A', 'Empty graph'),
 (10, {}, defaultdict(list, {}), {}, 'A', 'Empty graph'),
 (11, {}, defaultdict(list, {}), {}, 'A', 'Empty graph'),
 (12, {}, defaultdict(list, {}), {}, 'A', 'Empty graph'),
 (13,
  {'_1': [62.0, 85.0],
   '_2': [63.0, 87.0],
   '_3': [93.0, 97.0],
   '_4': [96.0, 99.0]},
  defaultdict(list,
              {'_1': ([('_1', '_2')], [('_2', '_1')]),
               '_2': ([('_2', '_1')], [('_1', '_2')]),
               '_3': ([('_3

In [11]:
train, valid, test = load_data()
params, accuracy   = find_optimal_params(train, valid)

['base', 'classmodel', 'count']
['base', 'classmodel', 'count']
['base', 'classmodel', 'count']


KeyboardInterrupt: 

params

In [7]:
accuracy

84.0

In [ ]:
import pickle

def read_chunk(name):    
    
    with open(name, 'rb') as fp:
        data = pickle.load(fp)       
        
    return data 

In [13]:
import time
train, valid, test = load_data()
tr=read_chunk('final-chunk-2')
predictions = []

start = time.time()
for x in range(len(test)):
    neighbors = getNeighbors(tr, test[x], k=1 , TAU_E= 2.2 , TAU_N= 1.3)
    result = getResponse(neighbors)
    predictions.append(result)
    #print('> predicted=' + repr(result) + ', actual=' + repr(test[x][-2]))

accuracy = getAccuracy(test, predictions)
print('Accuracy: ' + repr(accuracy) + ' %')
stop = time.time()
#print("Time: {:.1f}".format((stop-start))+' seconds')

C:\Users\VAIO\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\VAIO\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
C:\Users\VAIO\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
C:\Users\VAIO\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
C:\Users\VAIO\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
C:\Users\VAIO\An

Accuracy: 80.60000000000001 %


In [13]:
a=0
w=0
l=0
r=0

for graph in train:
    if graph[-2 ]=='A' :
        a+=1
    elif graph[-2 ]=='L' :
        l+=1
    elif graph[-2 ]=='W' :
        w+=1  
    elif graph[-2 ]=='R' :
        r+=1      
    

In [14]:
a

198

In [15]:
w

88

In [16]:
r

92

In [18]:
l

122

In [5]:
train, valid, test = load_data()

C:\Users\VAIO\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\VAIO\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
C:\Users\VAIO\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
C:\Users\VAIO\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
C:\Users\VAIO\Anaconda3\lib\site-packages\ipykernel_launcher.py:46: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
C:\Users\VAIO\An

In [13]:
train[286:408]

[(286,
  {'_1': [147.0, 125.0], '_2': [171.0, 140.0], '_3': [158.0, 142.0]},
  defaultdict(list,
              {'_1': ([('_1', '_3')], [('_3', '_1')]),
               '_2': ([('_2', '_3')], [('_3', '_2')]),
               '_3': ([('_3', '_1'), ('_3', '_2')],
                [('_1', '_3'), ('_2', '_3')])}),
  {('_1', '_3'): [1, [0.83957, 0.996491]],
   ('_3', '_1'): [1, [-0.83957, -2.145101]],
   ('_3', '_2'): [1, [-0.152057, -0.152649]],
   ('_2', '_3'): [1, [0.152057, 2.988943]]},
  'L',
  'Normal graph'),
 (287,
  {'_1': [149.0, 115.0],
   '_2': [193.0, 191.0],
   '_3': [161.0, 126.0],
   '_4': [173.0, 136.0],
   '_5': [186.0, 146.0],
   '_6': [200.0, 155.0],
   '_7': [197.0, 172.0]},
  defaultdict(list,
              {'_1': ([('_1', '_3')], [('_3', '_1')]),
               '_2': ([('_2', '_7')], [('_7', '_2')]),
               '_3': ([('_3', '_1'), ('_3', '_4')],
                [('_1', '_3'), ('_4', '_3')]),
               '_4': ([('_4', '_3'), ('_4', '_5')],
                [('_3',